In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=3fef0253ef6ff95a2bdb7742722846dfc29e444def98bef2ccc00cd84b4e8155
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

# Inicie a sessão do Spark
spark = SparkSession.builder.appName("ImportCSV").getOrCreate()

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving all_stocks_5yr.csv to all_stocks_5yr.csv


In [ ]:
file_path = "/content/all_stocks_5yr.csv"  # Substitua pelo caminho fornecido pelo Colab

df = spark.read.format("csv").option("header", "true").load(file_path)

In [ ]:
# Verificar os atributos do DataFrame
print(df.columns)

['date', 'open', 'high', 'low', 'close', 'volume', 'Name']


In [ ]:
# Verificar os tipos de dados das colunas
print(df.dtypes)

[('date', 'string'), ('open', 'string'), ('high', 'string'), ('low', 'string'), ('close', 'string'), ('volume', 'string'), ('Name', 'string')]


In [ ]:
# Verificar o esquema do DataFrame
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- Name: string (nullable = true)



In [ ]:
df.count()

619040

In [ ]:
df.show()

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28|13.49|13.63|13.39|13.43| 6143600| AAL|
|2013-03-01|13.37|13.95|13.32|13.61| 7376800| AAL|
|2013-03-04| 13.5|14.07|13.47| 13.9| 8174800| AAL|
|2013-03-05|14.01|14.05|13.71|1

In [ ]:
df.describe().show()

+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+------+
|summary|      date|             open|             high|              low|            close|           volume|  Name|
+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+------+
|  count|    619040|           619029|           619032|           619032|           619040|           619040|619040|
|   mean|      null|83.02333431454696|83.77831069346465|82.25609641375786|83.04376276476573|4321823.395568945|  null|
| stddev|      null| 97.3787690433231|  98.207518904464| 96.5074210580914|97.38974800165752|8693609.511967659|  null|
|    min|2013-02-08|             1.62|             1.69|              1.5|             1.59|                0|     A|
|    max|2018-02-07|           999.47|           999.75|           999.48|            999.6|           999994|   ZTS|
+-------+----------+-----------------+-----------------+

In [ ]:
# Pergunta 2: Quantos registros há na planilha para a ação da Apple (AAPL)?
valor_desejado = 'AAPL'
count = df.filter(df['Name'] == valor_desejado).count()
print("Número de registros com o valor", valor_desejado, ":", count)

Número de registros com o valor AAPL : 1259


In [ ]:
# Pergunta 3: Quantas empresas distintas têm registros nessa planilha?
coluna_alvo = 'Name'
count = df.select(coluna_alvo).distinct().count()
print("Número de registros distintos na coluna", coluna_alvo, ":", count)

Número de registros distintos na coluna Name : 505


In [ ]:
# Pergunta 4: Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?
total_count = df.count()
greater_count = df.filter(df['Close'] > df['Open']).count()
percentage = (greater_count / total_count) * 100
print("Frequência em que os valores de 'Close' são maiores que os de 'Open':", percentage, "%")

Frequência em que os valores de 'Close' são maiores que os de 'Open': 51.52171103644353 %


In [ ]:
# Pergunta 5: Qual o maior valor das ações da Apple (AAPL) na história?
from pyspark.sql.functions import col

# Converter a coluna "Close" para o tipo numérico
df = df.withColumn("Close", col("Close").cast("float"))

# Encontrar o maior valor das ações da Apple (AAPL) na coluna "Close"
nome_acao = 'AAPL'
maior_valor = df.filter(df['Name'] == nome_acao).agg({"Close": "max"}).collect()[0][0]
print("Maior valor da ação", nome_acao, ":", maior_valor)

Maior valor da ação AAPL : 179.25999450683594


In [ ]:
# Pergunta 6: Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.
nome_acao = 'PCLN'
desvio_padrao = df.filter(df['Name'] == nome_acao).agg({'Close': 'stddev'}).collect()[0][0]
print("Desvio padrão dos valores da ação", nome_acao, ":", desvio_padrao)

Desvio padrão dos valores da ação PCLN : 320.533474036385


In [ ]:
# Pergunta 6: Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.
nome_acao = 'AES'
desvio_padrao = df.filter(df['Name'] == nome_acao).agg({'Close': 'stddev'}).collect()[0][0]
print("Desvio padrão dos valores da ação", nome_acao, ":", desvio_padrao)

Desvio padrão dos valores da ação AES : 1.4871795692373793


In [ ]:
# Pergunta 6: Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.
nome_acao = 'GOOGL'
desvio_padrao = df.filter(df['Name'] == nome_acao).agg({'Close': 'stddev'}).collect()[0][0]
print("Desvio padrão dos valores da ação", nome_acao, ":", desvio_padrao)

Desvio padrão dos valores da ação GOOGL : 187.5738922672348


In [ ]:
# Pergunta 6: Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.
nome_acao = 'AMZN'
desvio_padrao = df.filter(df['Name'] == nome_acao).agg({'Close': 'stddev'}).collect()[0][0]
print("Desvio padrão dos valores da ação", nome_acao, ":", desvio_padrao)

Desvio padrão dos valores da ação AMZN : 282.50039426072794


In [ ]:
# Pergunta 7: Qual o dia com maior volume total de negociação da bolsa?
from pyspark.sql.functions import col

# Encontrar o dia com o maior volume total de negociação da bolsa
result = df.groupBy('date').agg({'volume': 'sum'}).orderBy(col('sum(volume)').desc()).first()
date_with_max_volume = result['date']
print("Dia com o maior volume total de negociação da bolsa:", date_with_max_volume)

Dia com o maior volume total de negociação da bolsa: 2015-08-24


In [ ]:
# Pergunta 8: Qual a ação mais negociada da bolsa, em volume de transações?
from pyspark.sql.functions import col

# Encontrar a ação mais negociada da bolsa em termos de volume de transações
result = df.groupBy('Name').agg({'volume': 'sum'}).orderBy(col('sum(volume)').desc()).first()
acao_mais_negociada = result['Name']
print("Ação mais negociada da bolsa em volume de transações:", acao_mais_negociada)

Ação mais negociada da bolsa em volume de transações: BAC


In [ ]:
# Pergunta 9: Quantas ações começam com a letra “A”?

count = df.filter(col('Name').startswith('A')).select('Name').distinct().count()
print("Número de ações únicas que começam com a letra 'A':", count)

Número de ações únicas que começam com a letra 'A': 59


In [ ]:
# Pergunta 10: Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

count = df.filter(col('high') == col('close')).count()
total_count = df.count()
frequency = (count / total_count) * 100
print("Frequência em que o preço mais alto do dia é igual ao preço de fechamento:", frequency, "%")

Frequência em que o preço mais alto do dia é igual ao preço de fechamento: 1.1986301369863013 %


In [ ]:
# Pergunta 10: Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?

from pyspark.sql.functions import col, abs

# Criar coluna de variação absoluta entre abertura e fechamento
df_with_diff = df.withColumn('diff', abs(col('open') - col('close')))

# Filtrar apenas registros da ação da Apple (AAPL)
df_aapl = df_with_diff.filter(col('Name') == 'AAPL')

# Ordenar por diferença absoluta em ordem decrescente e selecionar o primeiro registro
result = df_aapl.orderBy(col('diff').desc()).first()
date_with_max_diff = result['date']

print("Dia em que a ação da Apple mais subiu entre abertura e fechamento, de forma absoluta:", date_with_max_diff)

Dia em que a ação da Apple mais subiu entre abertura e fechamento, de forma absoluta: 2015-08-24


In [ ]:
# Pergunta 12: Em média, qual o volume diário de transações das ações da AAPL?


from pyspark.sql.functions import col

# Converter a coluna 'volume' para o tipo numérico
df = df.withColumn("volume", col("volume").cast("float"))

# Calcular a média do volume diário de transações das ações da AAPL
average_volume = df.filter(df['Name'] == 'AAPL').groupBy().avg('volume').collect()[0][0]
print("Média do volume diário de transações das ações da AAPL:", average_volume)

Média do volume diário de transações das ações da AAPL: 54047899.74821287


In [ ]:
# Pergunta 13: Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

# Selecionar registros únicos da coluna 'Name' e ordená-los por tamanho
unique_names = df.select('Name').distinct().orderBy('Name')

# Exibir os resultados
unique_names.show()

+----+
|Name|
+----+
|   A|
| AAL|
| AAP|
|AAPL|
|ABBV|
| ABC|
| ABT|
| ACN|
|ADBE|
| ADI|
| ADM|
| ADP|
| ADS|
|ADSK|
| AEE|
| AEP|
| AES|
| AET|
| AFL|
| AGN|
+----+
only showing top 20 rows



In [ ]:
# Filtrar registros com apenas 1 caractere na coluna 'Name' e contar o número de ocorrências
count_1_char = unique_names.filter(length('Name') == 1).count()

# Imprimir o resultado
print("Número de registros com apenas 1 caractere:", count_1_char)

Número de registros com apenas 1 caractere: 10


In [ ]:
# Filtrar registros com apenas 2 caracteres na coluna 'Name' e contar o número de ocorrências
count_2_char = unique_names.filter(length('Name') == 2).count()

# Imprimir o resultado
print("Número de registros com apenas 2 caracteres:", count_2_char)

Número de registros com apenas 2 caracteres: 50


In [ ]:
# Filtrar registros com apenas 3 caracteres na coluna 'Name' e contar o número de ocorrências
count_3_char = unique_names.filter(length('Name') == 3).count()

# Imprimir o resultado
print("Número de registros com apenas 3 caractere:", count_3_char)

Número de registros com apenas 3 caractere: 323


In [ ]:
# Filtrar registros com apenas 4 caracteres na coluna 'Name' e contar o número de ocorrências
count_4_char = unique_names.filter(length('Name') == 4).count()

# Imprimir o resultado
print("Número de registros com apenas 4 caractere:", count_4_char)

Número de registros com apenas 4 caractere: 117


In [ ]:
# Filtrar registros com apenas 5 caracteres na coluna 'Name' e contar o número de ocorrências
count_5_char = unique_names.filter(length('Name') == 5).count()

# Imprimir o resultado
print("Número de registros com apenas 5 caractere:", count_5_char)

Número de registros com apenas 5 caractere: 5


In [ ]:
# Pergunta 14: Qual a ação menos negociada da bolsa, em volume de transações?

from pyspark.sql.functions import col

# Encontrar a ação menos negociada da bolsa em termos de volume de transações
result = df.groupBy('Name').agg({'volume': 'sum'}).orderBy(col('sum(volume)')).first()
acao_menos_negociada = result['Name']
print("Ação menos negociada da bolsa em volume de transações:", acao_menos_negociada)

Ação menos negociada da bolsa em volume de transações: APTV
